#**CLASSIFICATION PNEUMONIE VIA TRANSFER LEARNING**

In [18]:
# installation des librairies nécessaires

!pip install keras
!pip install kagglehub
!pip install tensorflow
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [56]:
# import des librairies

import kagglehub
import seaborn as sns
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from google.colab import drive
import sys
import tensorflow
from keras.preprocessing import image_dataset_from_directory
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy, Recall, Precision, AUC
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

print("Importations terminées")

# constante
BATCH_SIZE = 32
IMG_SIZE = (224,224)


Importations terminées


Maintenant que les toutes les librairies sont installées et ont bien été importées, il est temps d'importer le jeu de données. Ce-dernier est récupéré sur Kaggle : https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
Il est composé de 3 dossiers (train, test et val) dans lesquels il y a pour chacuns d'eux 2 sous-dossiers (NORMAL et PNEUMONIA) contenant les radios thoraciques de patients ayant des poumons malades (pneumonie) ou des poumons sains.

In [25]:
# téléchargement des données venant de Kaggle
# à savoir qu'au téléchargement, les trois dossiers se trouvent dans un dossier "chest_xray" que l'on doit indiquer

chemin_telechargement = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
print(f"chemin du jeu de données : {chemin_telechargement}")

dossier_base = os.path.join(chemin_telechargement, "chest_xray")

chemin_train = os.path.join(dossier_base, "train")
chemin_test = os.path.join(dossier_base, "test")

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
chemin du jeu de données : /kaggle/input/chest-xray-pneumonia


Maintenant que le jeu de données est importer dans notre dossier Google Colab, nous allons utiliser la fonction "image_dataset_from_directory()" pour indiquer dans quel sous-dossier keras est sensé construire le jeu d'entraînement, de test et d'évaluation.

In [37]:
train_set = image_dataset_from_directory(
    directory=chemin_train,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    seed=123,
    subset="training"
)
val_set = image_dataset_from_directory(
    directory=chemin_train,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    seed=123,
    subset="validation"
)
test_set = image_dataset_from_directory(
    directory=chemin_test,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.
Found 624 files belonging to 2 classes.


Nous remarquons que notre dossier jeu d'entraînement contient 5 216 images tandis que le jeu de test en contient 624.

Nous allons à présent commencer la création du modèle. Le but de cet exercice est d'importer un modèle pré-entraîner sur un large volume de données (ici le modèle VGG16 entraîner sur la banque d'image "imagenet"), de freeze ce modèle pour créer le notre qui ne reconnaisse que les pneumonies et les cas sains et ainsi gagner en temps d'entraînement.


In [48]:
inputs = Input(shape=(224,224,3))
x = preprocess_input(inputs) # transforme l'input pour que VGG comprenne

baseModel = VGG16(include_top=False, weights="imagenet", input_tensor=x)
baseModel.trainable = False # on gèle les poids du modèle VGG16
x = baseModel.output # sortie du modèle de base
x = GlobalAveragePooling2D()(x)
x = Dense(units=128, activation="relu")(x)
x = Dropout(rate=0.2)(x) # on éteint 20% des neuronnes au hasard à chaque tour
outputs = Dense(units=1, activation="sigmoid")(x) # binaire car Pneumonie ou Normale

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack_1 (Stack)     │ (None, 224, 224,  │          0 │ get_item_3[0][0], │
│                     │ 3)                │            │ get_item_4[0][0], │
│                     │                   │            │ get_item_5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 224, 224,  │          0 │ stack_1[0][0]     │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 224, 224,  │      1,792 │ add_1[0][0]       │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 224, 224,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 112, 112,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 112, 112,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 112, 112,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 56, 56,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 56, 56,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 56, 56,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 56, 56,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 28, 28,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 14,780,481 (56.38 MB)

 Trainable params: 65,793 (257.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [49]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[
        BinaryAccuracy(name="accuracy"),
        Recall(name="recall"), # ne pas rater les malades IMPORTANT
        Precision(name="precision"),
        AUC(name="auc") # aire sous la courbe qui mesure la performance globale
    ]
)

In [51]:
total = nombre_normal + nombre_pneumonie
# dans notre jeu de donnée il y a beaucoup plus de cas de pneumonie que de cas normaux
# il faut donc pendant l'entraînement que le modèle calcule ses poids en fonction du nombre de cas
# "une erreur sur un cas normal compte triple"
# formule pour équilibrer les poids : poids = (1 / nombreClasses) * (total / 2)
weightFor1 = (1 / nombre_pneumonie) * (total / 2) # pneumonie
weightFor0 = (1 / nombre_normal) * (total / 2) # normal
class_weight = {0: weightFor0, 1: weightFor1} # dictionnaire pour classer les poids
print(f"Poids Classe 0 (Normal)   : {weightFor0:.2f}")
print(f"Poids Classe 1 (Pneumonie): {weightFor1:.2f}")

Poids Classe 0 (Normal)   : 1.94
Poids Classe 1 (Pneumonie): 0.67


In [52]:
# CALLBACK
# earlystopping : on s'arrête si le modèle n'apprend plus
# modelCheckpoint : on sauvegarde le meilleur état du modèle (perte la plus basse sur la validation)
# le dernier est parfois en sur-apprentissage

earlyStopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
# arrête la "val_loss" si le modèle n'apprend plus depuis 3 epochs

checkPoint = ModelCheckpoint(
    filepath="modelVGG16ClassificationPneumonie.keras",
    monitor="val_loss",
    save_best_only=True,
    mode="min",
    verbose=1
)

Pour des raisons de temps et de complications, nous n'allons pas entraîner le modèle dans Google Colab (environ une heure d'entraînement) mais je vous laisse le soin de dé-commenter les lignes ci-dessous pour l'entraînement vous-même.

Nous allons donc importer le modèle final déjà entraîner dans Google Colab.